In [11]:
import os
import shutil
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib.pyplot as plt

from PIL import Image

# Use COCO_train_test_split library

- multi-class인 COCO_dataset을 train/test 분할

- CLI에서 
    - !python cocosplit.py --having-annotations --multi-class -s 0.8 {/path/to/your/coco_annotations.json} {train.json} {test.json}

In [17]:
!mkdir -p data/labels_YOLO/train data/labels_YOLO/test
%cd ./data_preprocess/COCO_train_test

!python cocosplit.py --having-annotations --multi-class -s 0.8 ../../data/labels/train.json ../../data/labels_YOLO/train/train.json ../../data/labels_YOLO/test/test.json
%cd ../../

/home/yebu/ai_competitions/aifactory/fish_copy/data_preprocess/COCO_train_test
Saved 5040 entries in ../../data/labels_YOLO/train/train.json and 1261 in ../../data/labels_YOLO/test/test.json
/home/yebu/ai_competitions/aifactory/fish_copy


# Move images into train/test folder

In [18]:
!mkdir -p data/YOLO/train data/YOLO/test data/YOLO/train_test_all

In [ ]:
# install pycocotools
!pip install pycocotools

In [19]:
from pycocotools.coco import COCO

train_split_json = './data/labels_YOLO/train/train.json'
test_split_json  = './data/labels_YOLO/test/test.json'

coco_train_split = COCO(train_split_json)
coco_test_split  = COCO(test_split_json)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.21s)
creating index...
index created!


In [3]:
# Create train_data_info in dictionary

from collections import defaultdict

train_dict = defaultdict(list)
test_dict  = defaultdict(list)

for i in range(149821):
    try:
        # annotation_id를 기반으로 annotation dict의 전체 정보를 return
        ann_info = coco_train_split.loadAnns(i)[0]

        image_id = ann_info['image_id']
        category = ann_info['category_id']
        bbox     = ann_info['bbox']

        file_name = coco_train_split.loadImgs(image_id)[0]['file_name']

        train_dict[file_name].append((image_id, category, bbox))
    except:
        pass

for i in range(149821):
    try:
        # annotation_id를 기반으로 annotation dict의 전체 정보를 return
        ann_info = coco_test_split.loadAnns(i)[0]

        image_id = ann_info['image_id']
        category = ann_info['category_id']
        bbox     = ann_info['bbox']

        file_name = coco_test_split.loadImgs(image_id)[0]['file_name']

        test_dict[file_name].append((image_id, category, bbox))
    except:
        pass

In [4]:
cnt = 0

for i in train_dict.items():
    print(i)
    cnt += 1
    if cnt == 3:
        break
    
print("                      ")

cnt = 0

for i in test_dict.items():
    print(i)
    cnt += 1
    if cnt == 3:
        break

('train_0.png', [(1, 3, [37.98, 209.94, 369.19, 110.89])])
('train_186.png', [(284, 5, [96.59, 236.87, 427.01, 137.82])])
('train_194.png', [(297, 5, [26.18, 222.61, 409.5, 171.09])])
                      
('train_185.png', [(283, 5, [57.07, 247.17, 384.16, 130.69])])
('train_192.png', [(292, 5, [154.41, 254.3, 467.41, 134.65])])
('train_195.png', [(299, 5, [80.04, 228.95, 373.07, 157.62])])


In [5]:
os.getcwd()

'/home/yebu/ai_competitions/aifactory/fish_copy'

In [6]:
# Copy images into yolo_train / yolo_test

train_img_list = list(train_dict.keys())
test_img_list  = list(test_dict.keys())

data_origin = './data/train/'

train_dir = './data/YOLO/train'
test_dir  = './data/YOLO/test'


for idx, i in enumerate([train_img_list, test_img_list]):
    
    for img in tqdm(i):
        from_file_path = data_origin + img

        if idx == 0:
            to_file_path = train_dir
        else:
            to_file_path = test_dir

        shutil.copy(from_file_path, to_file_path)

100%|██████████| 1169/1169 [00:01<00:00, 928.01it/s] 


In [20]:
%cd data_preprocess
"""
!git clone https://github.com/ssaru/convert2Yolo.git
"""


%cd convert2Yolo
"""
!pip install -r requirements.txt
"""

/USER/mnc/aifactory_fish/fish_copy/data_preprocess
/USER/mnc/aifactory_fish/fish_copy/data_preprocess/convert2Yolo


/root/anaconda3/envs/fish/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'\n!pip install -r requirements.txt\n'

In [7]:
labels = ['농어', '베스', '숭어', '강준치', '블루길', '잉어', '붕어', '누치']


with open('fish.names', 'w+', encoding='UTF-8') as f:
    for i in labels:
        f.write(i + '\n')

In [9]:
os.getcwd()

'/home/yebu/ai_competitions/aifactory/fish_copy'

In [10]:
%cd /home/yebu/ai_competitions/aifactory/fish_copy/data_preprocess/convert2Yolo

!python3 example.py --datasets COCO --img_path ../../data/YOLO/train --label ../../data/labels_YOLO/train/train.json --convert_output_path ../../data/YOLO/train --img_type ".png" --manifest_path ../../data/YOLO/train_test_all/train.txt --cls_list_file ../../data_preprocess/convert2Yolo/fish.names

!python3 example.py --datasets COCO --img_path ../../data/YOLO/test --label ../../data/labels_YOLO/test/test.json --convert_output_path ../../data/YOLO/test --img_type ".png" --manifest_path ../../data/YOLO/train_test_all/test.txt --cls_list_file ../../data_preprocess/convert2Yolo/fish.names

%cd /home/yebu/ai_competitions/aifactory/fish_copy

/home/yebu/ai_competitions/aifactory/fish_copy/data_preprocess/convert2Yolo



/home/yebu/anaconda3/envs/med_com/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


COCO Parsing:  |████████████████████████████████████████| 100.0% (5040/5040)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (4683/4683)  Complete████---------------| 64.2% (3007/4683)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (4683/4683)  Complete


COCO Parsing:  |████████████████████████████████████████| 100.0% (1261/1261)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (1169/1169)  Complete███--------------------------| 37.4% (437/1169)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (1169/1169)  Complete

/home/yebu/ai_competitions/aifactory/fish_copy


In [ ]:
# COCO에서 object가 있는 image만 골라낸 후 coco에서 yolo로 converting이 끝남. 